### SocialAL Models
# Data simulation and parameter recovery - multiple subjects
KLS 8.30.19; update 3.5.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [3]:
run common_functions.ipynb

In [4]:
run double_alpha_decay_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    decay = params[3]
    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # after choice, update probability
        if choice != 1:
            Probs[t] = update_prob(response, Probs[t], a_gain, a_loss) 
        # then update value
        EVs[t] = update_value(Probs[t]) 
        
        # decay prob and value for other possible trustees 
        options = [0, 1, 2]
        if choice == 2 or choice == 3 or choice == 4:
            options.pop(t)
        for x in options:
            Probs[x] = Probs[x] + decay * (0.5 - Probs[x])
            EVs[x] =  update_value(Probs[x]) 

    data = {'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses}    
    return data

In [8]:
#sim_data(15,[.2, .3, 2, .4])

### New function to sim data for multiple subjects

In [9]:
def sim_mult_subject_data(n, tn, params):
    data = pd.DataFrame(columns= ['Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Alpha_gain', 'Alpha_loss','Beta', 'Decay'])
    for x in range(1, n+1):
        sdt = sim_data(tn, params)
        sdt = pd.DataFrame.from_dict(sdt)
        sdt['Subject'] = [x for t in range(0,tn*3)]
        sdt['Alpha_gain'] = params[0]
        sdt['Alpha_loss'] = params[1]
        sdt['Beta'] = params[2]
        sdt['Decay'] = params[3]
        data = pd.concat([data,sdt])
    return data

In [10]:
# check sim multisubs
#data = sim_mult_subject_data(100, 15, [.2,.3, 2, .4])
#data

### Create param space

In [11]:
# create parameter space
alpha_gains = [round(x * 0.2,1) for x in range(0,6)]
alpha_loses = [round(x * 0.2,1) for x in range(0,6)]
betas = [x * 4 for x in range(0,6)]
decays = [round(x * 0.2,1) for x in range(0,6)]
combos = list(itertools.product(alpha_gains, alpha_loses, betas, decays))
#for r in combos: print(r)

(0.0, 0.0, 0, 0.0)
(0.0, 0.0, 0, 0.2)
(0.0, 0.0, 0, 0.4)
(0.0, 0.0, 0, 0.6)
(0.0, 0.0, 0, 0.8)
(0.0, 0.0, 0, 1.0)
(0.0, 0.0, 4, 0.0)
(0.0, 0.0, 4, 0.2)
(0.0, 0.0, 4, 0.4)
(0.0, 0.0, 4, 0.6)
(0.0, 0.0, 4, 0.8)
(0.0, 0.0, 4, 1.0)
(0.0, 0.0, 8, 0.0)
(0.0, 0.0, 8, 0.2)
(0.0, 0.0, 8, 0.4)
(0.0, 0.0, 8, 0.6)
(0.0, 0.0, 8, 0.8)
(0.0, 0.0, 8, 1.0)
(0.0, 0.0, 12, 0.0)
(0.0, 0.0, 12, 0.2)
(0.0, 0.0, 12, 0.4)
(0.0, 0.0, 12, 0.6)
(0.0, 0.0, 12, 0.8)
(0.0, 0.0, 12, 1.0)
(0.0, 0.0, 16, 0.0)
(0.0, 0.0, 16, 0.2)
(0.0, 0.0, 16, 0.4)
(0.0, 0.0, 16, 0.6)
(0.0, 0.0, 16, 0.8)
(0.0, 0.0, 16, 1.0)
(0.0, 0.0, 20, 0.0)
(0.0, 0.0, 20, 0.2)
(0.0, 0.0, 20, 0.4)
(0.0, 0.0, 20, 0.6)
(0.0, 0.0, 20, 0.8)
(0.0, 0.0, 20, 1.0)
(0.0, 0.2, 0, 0.0)
(0.0, 0.2, 0, 0.2)
(0.0, 0.2, 0, 0.4)
(0.0, 0.2, 0, 0.6)
(0.0, 0.2, 0, 0.8)
(0.0, 0.2, 0, 1.0)
(0.0, 0.2, 4, 0.0)
(0.0, 0.2, 4, 0.2)
(0.0, 0.2, 4, 0.4)
(0.0, 0.2, 4, 0.6)
(0.0, 0.2, 4, 0.8)
(0.0, 0.2, 4, 1.0)
(0.0, 0.2, 8, 0.0)
(0.0, 0.2, 8, 0.2)
(0.0, 0.2, 8, 0.4)
(0.0, 0.2, 8,

### Simulate data

In [12]:
data = pd.DataFrame(columns= ['Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Alpha_gain', 'Alpha_loss','Beta', 'Decay'])
for p in range(len(combos)):
    dt = sim_mult_subject_data(10, 15, combos[p])
    dt['Subject'] = dt['Subject'] + p*10
    data = pd.concat([data, dt])
data

data.to_csv(path_or_buf = '../../output/simulation/sim_2alpha_decay_model_data.csv', index = False)

### New function to fit model to multiple subjects

In [13]:
def model_fit_mult_subject(data):
    pdt = pd.DataFrame(columns = ['Subject', 'a_gain', 'a_loss', 'beta', 'decay'])
    a = pd.unique(data['Subject'])
    print('Number of Subs: ', len(a))
    for sub in range(1,len(a)+1):
        print('Subject: ', sub)
        df = data[data['Subject']==sub]
        dt = df.to_dict()
        params = model_fit(dt)
        line = {'Subject': sub, 'a_gain': params[0], 'a_loss': params[1], 'beta':params[2], 'decay':params[3]}
        pdt = pdt.append(line, ignore_index=True)   
    return(pdt)


In [14]:
precover = model_fit_mult_subject(data)
precover.to_csv(path_or_buf = '../../output/simulation/sim_2alpha_decay_model_fit.csv', index = False)

Number of Subs:  12960
Subject:  1
Subject:  2
Subject:  3
Subject:  4
Subject:  5
Subject:  6
Subject:  7
Subject:  8
Subject:  9
Subject:  10
Subject:  11
Subject:  12
Subject:  13
Subject:  14
Subject:  15
Subject:  16
Subject:  17
Subject:  18
Subject:  19
Subject:  20
Subject:  21
Subject:  22
Subject:  23
Subject:  24
Subject:  25
Subject:  26
Subject:  27
Subject:  28
Subject:  29
Subject:  30
Subject:  31


KeyboardInterrupt: 

In [ ]:
print(precover)